# Feasibility Problems or Solving Systems of Nonlinear Equations

When we use an optimization algorithm and are trying to describe the quality of a given design point in that design space, there are two adjectives that come up often. Let's define them:

* *Feasibility*: This means that the point lies within the feasible region of the design space, i.e. no constraints are violated. This is also referred to as "primal feasibility".

* *Optimality*: This means that if we move an infinitesimal amount in **any** *feasible direction* (a direction that doesn't cause us to violate constraints if we take this infinitesimal step), we won't find a point that is better. In lay terms, "this point is better than or equal to all of its neighbors". This is also referred to as "dual feasibility".

Clearly, the goal of our algorithm is to find a point that is both feasible and optimal. Is this problem we're asking [well-posed](https://en.wikipedia.org/wiki/Well-posed_problem)? In other words:

1. Does a solution exist?

2. Is that solution unique?

3. Is the solution stable w.r.t. perturbations in problem parameters?

Well, in the case of a continuous, unimodal problem, the answer is generally *yes*.

Sometimes, however, we're only concerned with feasibility and not optimality. Another way to state this would be that we might have a scenario where all feasible solutions are equally optimal.

How would we encode this in AeroSandbox?

We could either write:

* `opti.minimize(0)`, which explicitly says that the performance of any feasible design is zero everywhere, and hence any feasible solution is equally optimal. We could also replace `0` with any other constant value.

* We could simply not write any kind of `opti.minimize()` statement, in which case the solver will default to `opti.minimize(0)` (effectively, no objective function.)

## Feasibility Problems in Design

Feasibility problems can come up in the case of engineering design, albeit not often. With feasibility problems, the problem is often not well-posed, because the solution may not be unique. That doesn't mean it's unsolvable; it simply means that it's difficult to tell *a priori* what our solution will be.

A code example of this follows. Here, we pose the trivial problem of finding some value of $x$ that satisfies $1 < x < 2$. We naively guess that $x = 5$:

In [3]:
import aerosandbox as asb
import aerosandbox.numpy as np

opti = asb.Opti()

x = opti.variable(init_guess=5)

opti.subject_to([
    x > 1,
    x < 2,
])

sol = opti.solve(verbose=False)

print(f"x_optimal = {sol.value(x)}")

x_optimal = 1.5274119593606093


So, we got a feasible solution! However, notice that the solution is some apparently-random (but deterministic) value between 1 and 2; it is not simply the value of the initial guess projected directly onto the nearest boundary of the feasible space (which would have given us $x \approx 2$).

In fact, here, there are infinite solutions arranged on a line between $x=1$ and $x=2$. All solutions are adjacent - take an infinitesimal step from any optimal point, and you'll get another optimal point.

This same thing can happen in higher dimensions, though it can be less obvious because the "lines" or "hyperplanes" of optimality are not necessarily axis-aligned. Because of that, when you suspect you have a design problem that you expect has this kind of geometry (an infinite number of adjacent optimal solutions), you might try adding a slight regularization term (such as a very weak quadratic penalty term) in order to "select" a single solution from all of the optima.

This can be a great tool for understanding the design space better.

## Solving Nonlinear Systems of Equations

This idea of a "feasibility problem" can also come up when solving a system of nonlinear equations. Here, we solve a system of nonlinear equations implicitly by setting their governing equations as constraints. The natural extension of this idea is something called Simultaneous Analysis and Design (SAND), but that's a bit beyond scope for just this lesson.

Let's give an example of solving a system of nonlinear equations:

$ y = x^2 $

$ y^2 = 18 - x $

This system of equations turns out to have two solutions: $(x, y) = (2, 4) = (-2.118, 4.485)$. Let's find them with AeroSandbox:

In [4]:
opti = asb.Opti()

x = opti.variable(init_guess=5)
y = opti.variable(init_guess=5)

opti.subject_to([
    y == x ** 2,
    y ** 2 == 18 - x
])

sol = opti.solve(verbose=False)

print(f"x_optimal = {sol.value(x)}")
print(f"y_optimal = {sol.value(y)}")

x_optimal = 2.000000000004754
y_optimal = 3.99999999999944


So, we found a solution for our governing equations.

We could find the other solution by trying a different initial guess:

In [7]:
opti = asb.Opti()

x = opti.variable(init_guess=-1)
y = opti.variable(init_guess=1)

opti.subject_to([
    y == x ** 2,
    y ** 2 == 18 - x
])

sol = opti.solve(verbose=False)

print(f"x_optimal = {sol.value(x)}")
print(f"y_optimal = {sol.value(y)}")

x_optimal = -2.117850972326501
y_optimal = 4.485292740984305


And, some initial guesses won't converge (typically, this is due to a sign flip in the constraints jacobian, driving the solution in the wrong direction):

In [11]:
opti = asb.Opti()

x = opti.variable(init_guess=-1)
y = opti.variable(init_guess=-1)

opti.subject_to([
    y == x ** 2,
    y ** 2 == 18 - x
])

try:
    sol = opti.solve()
except RuntimeError as e:
    print(e)
    print(f"x_last = {opti.debug.value(x)}")
    print(f"y_last = {opti.debug.value(y)}")


This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 0

Anyway, the basic idea is that if you're solving a system of nonlinear equations, you are solving a feasibility problem. With certain assumptions, this feasibility problem is well-posed even without adding an objective function with `opti.minimize()`. These assumptions:

* The constraint jacobian matrix is square and full-rank. What this means in non-math-speak:

    * You have the same number of variables and equality constraints. (For simplicity, assume we have no inequality constraints.)

    * None of your equality constraints are linearly dependent.

This is usually the case when solving a system of nonlinear equations - if it is not, you don't have a unique solution anyway, regardless of what method you're using to solve your problem.

(Side note: One cool thing about solving systems of nonlinear equations with an optimization approach: it's a very natural transition between solving well-posed systems of equations and finding solutions to underdetermined systems that minimize some error metric. This is sort of extending the idea of converting a matrix inverse into a Moore-Penrose pseudoinverse, but a more natural and extensible way.)